### Linking to the google drive to access any data or files

In [ ]:
# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

# Add the custom module path to sys.path to allow for imports from that directory
import sys
sys.path.append('/content/drive/MyDrive/main/src/')

Mounted at /content/drive


### Importing the required libraries

In [ ]:
# Automatically reload modules when they change
%load_ext autoreload
%autoreload 2

# Import custom modules
from model import initialise_model
from utils import set_requires_grad, save_model
from data import load_data

# Import standard libraries
import os
import random

# Import data processing and visualisation libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
import warnings

# Import PyTorch and torchvision libraries
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as td
import torchvision.transforms as T
from torchvision.io import read_image
from torchvision import datasets, models, transforms

### Setting up the seed and figure size

In [ ]:
# Configure Matplotlib to automatically adjust the figure size when saving plots
plt.rcParams["savefig.bbox"] = 'tight'

# Set a fixed seed for reproducibility
seed = 26
random.seed(seed)  # Seed for random module
torch.manual_seed(seed)  # Seed for PyTorch
torch.backends.cudnn.deterministic = True  # Ensures reproducibility in cuDNN
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)  # Seed for all GPUs if CUDA is available
np.random.seed(seed)  # Seed for NumPy

In [ ]:
# Initialise parameters for model and data
num_classes = 3
batch_size = 32
num_workers = 12
norm_arr = ([0.5159, 0.5159, 0.5159], [0.2554, 0.2554, 0.2554])
feature_extract = False
use_pretrained = None
input_size = 256
images_dir = "/content/drive/MyDrive/data/covid_pneumonia"
class_names = ['COVID', 'Normal', 'Pneumonia']

# Load data
data_loaders = load_data(images_dir, batch_size=batch_size, input_size=input_size, norm_arr=norm_arr, num_workers=num_workers)

# Function to load a model and send it to the device
def load_model(model_path, device):
    model = torch.load(model_path, map_location=device)
    model = model.to(device)
    return model

# Load pretrained models
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_paths = [
    '/content/drive/MyDrive/models/densenet121_50_model_weights.pth',
    '/content/drive/MyDrive/models/resnet34_50_model_weights.pth',
    '/content/drive/MyDrive/models/mobile_net_v3_large_50_model_weights.pth',
    '/content/drive/MyDrive/models/efficient_net_b1_50_model_weights.pth',
    '/content/drive/MyDrive/models/densenet121_tl_50_model_weights.pth'
]

models = [load_model(path, device) for path in model_paths]

<ipython-input-4-d919c48543a5>:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path, map_location=device)


### Setting up the Grad-CAM class

In [ ]:
# Grad-CAM class definition
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model.eval()  # Set the model to evaluation mode
        self.featuremaps = []
        self.gradients = []

        # Register hooks to capture gradients and feature maps
        target_layer.register_forward_hook(self.save_featuremaps)
        target_layer.register_backward_hook(self.save_gradients)

    def save_featuremaps(self, module, input, output):
        self.featuremaps.append(output)  # Save forward pass feature maps

    def save_gradients(self, module, grad_input, grad_output):
        self.gradients.append(grad_output[0])  # Save backward pass gradients

    def get_cam_weights(self, grads):
        return np.mean(grads, axis=(1, 2))  # Compute weights by averaging gradients

    def __call__(self, image, label=None):
        preds = self.model(image)  # Forward pass
        self.model.zero_grad()  # Zero the gradients

        if label is None:
            label = preds.argmax(dim=1).item()  # Get the label with the highest score

        preds[:, label].backward()  # Backward pass for the specific label

        # Get the gradients and feature maps
        featuremaps = self.featuremaps[-1].cpu().data.numpy()[0, :]
        gradients = self.gradients[-1].cpu().data.numpy()[0, :]

        weights = self.get_cam_weights(gradients)  # Compute weights
        cam = np.zeros(featuremaps.shape[1:], dtype=np.float32)

        # Combine the feature maps using the weights to create the CAM
        for i, w in enumerate(weights):
            cam += w * featuremaps[i]

        cam = np.maximum(cam, 0)  # Apply ReLU
        cam = cv2.resize(cam, image.shape[-2:][::-1])  # Resize CAM to image size
        cam = cam - np.min(cam)  # Normalise CAM
        cam = cam / np.max(cam)
        return label, cam

### Generating the Grad-CAM visualisations

In [ ]:
# Utility function to deprocess the image for visualisation
def deprocess_image(image):
    image = image.cpu().numpy()
    image = np.squeeze(np.transpose(image[0], (1, 2, 0)))

    image = image * np.array((0.2554, 0.2554, 0.2554)) + np.array((0.5159, 0.5159, 0.5159))
    image = image.clip(0, 1)
    return image

# Function to apply a mask to the image for visualisation
def apply_mask(image, mask):
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    cam = heatmap + np.float32(image)
    cam = cam / np.max(cam)
    return np.uint8(255 * cam)

# Function to get one sample image from each class
def get_sample_images(data_loader, num_samples=2):
    """ Get two sample from each class: COVID, Normal, and Pneumonia """
    samples = {class_name: [] for class_name in class_names}
    for images, labels in data_loader:
        for img, lbl in zip(images, labels):
            class_name = class_names[lbl]
            if len(samples[class_name]) < num_samples:
                samples[class_name].append((img, lbl))
        if all(len(samples[class_name]) >= num_samples for class_name in class_names):
            break
    return samples

# Get one image from each class
sample_images = get_sample_images(data_loaders['test'], num_samples=2)

# Suppress warnings
warnings.filterwarnings("ignore")

# Visualise Grad-CAM for each model using sample images
for class_name in class_names:
    for img_idx in range(2):
      image, label = sample_images[class_name][img_idx]
      image1 = image.to(device)
      label1 = label.to(device)

      image1 = image1.view(1, 3, 256, 256)  # Reshape image for model input

      cams = []

      # Target layers based on model architectures
      target_layers = [
          models[0].features[-1],        # DenseNet121
          models[1].layer4[-1],          # ResNet34
          models[2].features[-1],        # MobileNetV3
          models[3].features[-1],        # EfficientNet B1
          models[4].features[-1],        # DenseNet121 Transfer Learning
      ]

      # Generate Grad-CAM for each model
      for model, target_layer in zip(models, target_layers):
          cam_obj = GradCAM(model=model, target_layer=target_layer)
          _, cam = cam_obj(image1, label1)
          cams.append(cam)

      # Deprocess and apply masks
      image1 = deprocess_image(image1)
      masked_images = [apply_mask(image1, cam)[:, :, ::-1] for cam in cams]

      # Plot original image and CAMs
      fig, axes = plt.subplots(1, len(models) + 1, figsize=(24, 6))
      plt.setp(axes, xticks=[], yticks=[])

      axes[0].imshow(image1)
      axes[0].set_xlabel(f"{class_name} (ORIGINAL)")

      for ax, img, model_name in zip(axes[1:], masked_images, ['DENSENET', 'RESNET', 'MOBILENET', 'EFFICIENTNET', 'DENSENET TL']):
          ax.imshow(img)
          ax.set_xlabel(model_name)

      plt.subplots_adjust(wspace=0.00, hspace=0.00)
      plt.show()

Output hidden; open in https://colab.research.google.com to view.